# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Загрузка данных

In [1]:
# импорт библиотек
import pandas as pd
# модели
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
pd.options.mode.chained_assignment = None

In [2]:
# чтение файла и печать общей информации
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
# получение первых 5 строк таблицы
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Данные уже подготовлены для работы с моделями. Нет пропусуов и дубликтов.

Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Далее будет необходимо поделить данные на выборки и подготовить признаки для обучения моделей.

## 2. Разделение данных на выборки

In [4]:
# разделение на 2 выборки (обучающая и валидационная)
df_train, df_valid = train_test_split(df, test_size=0.1, random_state=12345)

# размер таблицы
df.shape

(3214, 5)

In [5]:
print('Кол-во строк в обучающей выборке:', df_train.shape[0])
print('Кол-во строк в валидационной выборке:', df_valid.shape[0])

Кол-во строк в обучающей выборке: 2892
Кол-во строк в валидационной выборке: 322


In [6]:
# добавил 3ю выборку для теста, того же размера, что и валидационная
df_test_size = df_valid.shape[0] / df_train.shape[0]
df_train, df_test = train_test_split(df_train, test_size=df_test_size, random_state=12345)

print('Кол-во строк в обучающей выборке:', df_train.shape[0])
print('Кол-во строк в валидационной выборке:', df_valid.shape[0])
print('Кол-во строк в тестовой выборке:', df_test.shape[0])

Кол-во строк в обучающей выборке: 2570
Кол-во строк в валидационной выборке: 322
Кол-во строк в тестовой выборке: 322


Валидационная и тестовая выпорка имеют одинаковые пропорции по 10% от DF.

Далее выделим целевой признак  и признаки для обучения модели. 

In [7]:
# разделение на обучающие признаки и целевые

features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

На данном этапе подготовили данные для обучения, проверки и тестирования моделей.

Поделил Датасет на три выборки:

- обучающая df_train,
- валидационая df_valid,
- тестовая df_test.

Валидационная и тестовая выпорка имеют одинаковые пропорции по 10% от DF.

Также, подготовили входные данные для обучения моделей.

## 3. Исследование модели

Перед нами стоит задача классификации, поэтому проверим 3 модели:

- DecisionTreeClassifier
- RandomForestClassifier
- LogisticRegression

### 3.1 DecisionTreeClassifier

In [8]:
# проведем перебор параметров и обучение модели с помледующей проверкой результатов
for depth in range(1, 10):
    model_DecisionTree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_DecisionTree.fit(features_train, target_train)
    predictions = model_DecisionTree.predict(features_train)
    print("max_depth =", depth, ": ")
    print("Результат проверки на обучающей  выборке", accuracy_score(target_train, predictions))
    predictions = model_DecisionTree.predict(features_valid)
    print("Результат проверки на валидауионной выборке", accuracy_score(target_valid, predictions))
    print()

max_depth = 1 : 
Результат проверки на обучающей  выборке 0.7509727626459144
Результат проверки на валидауионной выборке 0.7546583850931677

max_depth = 2 : 
Результат проверки на обучающей  выборке 0.7817120622568093
Результат проверки на валидауионной выборке 0.7763975155279503

max_depth = 3 : 
Результат проверки на обучающей  выборке 0.7972762645914397
Результат проверки на валидауионной выборке 0.7795031055900621

max_depth = 4 : 
Результат проверки на обучающей  выборке 0.809727626459144
Результат проверки на валидауионной выборке 0.7670807453416149

max_depth = 5 : 
Результат проверки на обучающей  выборке 0.8202334630350194
Результат проверки на валидауионной выборке 0.7701863354037267

max_depth = 6 : 
Результат проверки на обучающей  выборке 0.8280155642023346
Результат проверки на валидауионной выборке 0.7608695652173914

max_depth = 7 : 
Результат проверки на обучающей  выборке 0.8455252918287938
Результат проверки на валидауионной выборке 0.7701863354037267

max_depth = 8 

In [9]:
# авоматизизированный перебор параметрров для проверки результата с помощью GridSearchCV
my_list = []
for i in range(1, 500):
    my_list.append(i)
parameters = {'max_depth': my_list}
gs = GridSearchCV(DecisionTreeClassifier(random_state=12345), 
                  parameters,
                  scoring='accuracy')

# обучение
gs.fit(features_train, target_train)
# просмотр лучших параметров
gs.best_params_

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'max_depth': 9}

In [10]:
# лучший результат
gs.best_score_

0.8

In [11]:
# настройка модели исходя из тестов
model_DecisionTree = DecisionTreeClassifier(random_state=12345, max_depth=7)
model_DecisionTree.fit(features_train, target_train)
predictions_valid_DT = model_DecisionTree.predict(features_valid)
accuracy_score(target_valid, predictions_valid_DT)

0.7701863354037267

Ручной подбор параметров и GridSearchCV выявили что наилучший результат при max_depth = от 7 до 9.

Выберем параметр 7, так как он показывает лучший результат на валидационной выборке 0.77

### 3.2 RandomForestClassifier

In [12]:
# проведем перебор параметров и обучение модели с помледующей проверкой результатов
for estim in range(1, 11):
    model_RandomForest = RandomForestClassifier(random_state=12345, n_estimators=estim)
    model_RandomForest.fit(features_train, target_train)       
    predictions = model_RandomForest.predict(features_train)
    print("n_estimators =", estim, ": ")
    print("Результат проверки на обучающей  выборке", accuracy_score(target_train, predictions))
    predictions = model_RandomForest.predict(features_valid)
    print("Результат проверки на валидауионной выборке", accuracy_score(target_valid, predictions))
    print()

n_estimators = 1 : 
Результат проверки на обучающей  выборке 0.9089494163424124
Результат проверки на валидауионной выборке 0.7236024844720497

n_estimators = 2 : 
Результат проверки на обучающей  выборке 0.909727626459144
Результат проверки на валидауионной выборке 0.7515527950310559

n_estimators = 3 : 
Результат проверки на обучающей  выборке 0.9529182879377431
Результат проверки на валидауионной выборке 0.7515527950310559

n_estimators = 4 : 
Результат проверки на обучающей  выборке 0.9521400778210116
Результат проверки на валидауионной выборке 0.782608695652174

n_estimators = 5 : 
Результат проверки на обучающей  выборке 0.9743190661478599
Результат проверки на валидауионной выборке 0.7701863354037267

n_estimators = 6 : 
Результат проверки на обучающей  выборке 0.9684824902723735
Результат проверки на валидауионной выборке 0.7763975155279503

n_estimators = 7 : 
Результат проверки на обучающей  выборке 0.9809338521400778
Результат проверки на валидауионной выборке 0.779503105590

In [13]:
# авоматизизированный перебор параметрров для проверки результата с помощью GridSearchCV
my_list = []
for i in range(1, 20):
    my_list.append(i)
parameters = {'max_depth': [3, 4, 7],
             'n_estimators' : my_list
             }

gs = GridSearchCV(RandomForestClassifier(random_state=12345), 
                  parameters,
                  scoring='accuracy')

# обучение
gs.fit(features_train, target_train)
# просмотр лучших параметров
gs.best_params_

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'max_depth': 7, 'n_estimators': 10}

In [14]:
# лучший результат
gs.best_score_

0.8062256809338522

In [15]:
# настройка модели исходя из тестов
model_RandomForest = RandomForestClassifier(random_state=12345, max_depth=7, n_estimators=4)
model_RandomForest.fit(features_train, target_train)
predictions_valid_RF = model_RandomForest.predict(features_valid)
accuracy_score(target_valid, predictions_valid_RF)

0.7888198757763976

Ручной подбор параметров и GridSearchCV выявили что наилучший результат при n_estimators = от 4 до 10.

Выберем параметр 7, так как он показывает лучший результат на валидационной выборке 0.78

### 3.3 LogisticRegression

In [16]:
model_LogisticRegression = LogisticRegression(random_state=12345)
model_LogisticRegression.fit(features_train, target_train)
predictions_valid_LR = model_LogisticRegression.predict(features_valid)
accuracy_score(target_valid, predictions_valid_LR)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7018633540372671

Логическая регресия показывает результат 0,70

## 4. Проверка модели на тестовой выборке

In [17]:
# Проверка рещуьтатов на тестовой выборке
predictions_test_DT = model_DecisionTree.predict(features_test)
accuracy_score(target_test, predictions_test_DT)

0.7950310559006211

In [18]:
# Проверка рещуьтатов на тестовой выборке
predictions_test_RF = model_RandomForest.predict(features_test)
accuracy_score(target_test, predictions_test_RF)

0.8167701863354038

In [19]:
# Проверка рещуьтатов на тестовой выборке
predictions_test_LR = model_LogisticRegression.predict(features_test)
accuracy_score(target_test, predictions_test_LR)

0.7018633540372671

Проведя проверку модели на валидационно и тестовой выборке, лучший результат показвыаает модель RandomForest 

## 5. Проверьте модели на адекватность

In [20]:
# проверка на адекватность DummyClassifier 
dummy_clf =  DummyClassifier(strategy= "most_frequent") 
dummy_clf.fit(features_train, target_train)

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

In [21]:
# проверка на валидационной выборке
dummy_clf.score(features_valid,  target_valid)

0.7111801242236024

In [22]:
# проверка на тестовой выборке
dummy_clf.score(features_test,  target_test)

0.6956521739130435

Проверка на адекватность показала,что результат работы обченной модели лучше случайных результатов.

Лучший результат показвыаает модель RandomForest рещультат на тестовой выборке 0.81, на валидационной 0,78

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
